In [2]:
# !pip install pandas

In [3]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [4]:
url = 'https://www.tuhocielts.vn/tai-hon-3000-tu-tieng-anh-thong-dung-theo-chu-de-pdf-free/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}


response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
content_div = soup.find('div', class_='entry-content')
text_vocab = []
current_title = ""

In [6]:

for elem in content_div.find_all(['h3', 'p']):
    if elem.name == 'h3':
        raw_title = elem.get_text(strip=True)
        # Lấy phần sau từ "chủ đề" (không phân biệt hoa thường)
        match_title = re.search(r"(chủ đề.*)", raw_title, re.IGNORECASE)
        current_title = match_title.group(1).strip() if match_title else raw_title
    elif elem.name == 'p':
        lines = elem.get_text().split('\n')
        for line in lines:
            line = line.strip()
            if not line or ':' not in line:
                continue

            match = re.match(r"""^
                ([\w\-\s]+)               # Từ tiếng Anh
                (?:\s*/([^/]+)/)?        # Phiên âm (tùy chọn)
                (?:\s*\(([^)]+)\))?      # Loại từ (tùy chọn)
                \s*:\s*(.+)$             # Nghĩa tiếng Việt
            """, line, re.VERBOSE)

            if match:
                word = match.group(1).strip()
                ipa = f"/{match.group(2).strip()}/" if match.group(2) else ""
                word_type = match.group(3).strip() if match.group(3) else ""
                meaning = match.group(4).strip()
                
                text_vocab.append({
                    "topic": current_title,
                    "english": word,
                    "ipa": ipa,
                    "type": word_type,
                    "vietnamese": meaning
                })


In [7]:
# Ghi ra file JSON
# with open("vocabulary.json", "w", encoding="utf-8") as f:
#     json.dump(text_vocab, f, ensure_ascii=False, indent=4)

In [8]:
pdf_vocab = pd.DataFrame(text_vocab)

In [9]:
pdf_vocab['topic']

0                           chủ đề gia đình
1                           chủ đề gia đình
2                           chủ đề gia đình
3                           chủ đề gia đình
4                           chủ đề gia đình
                       ...                 
320    3.33. Từ vựng tiếng Anh về quê hương
321    3.33. Từ vựng tiếng Anh về quê hương
322    3.33. Từ vựng tiếng Anh về quê hương
323    3.33. Từ vựng tiếng Anh về quê hương
324    3.33. Từ vựng tiếng Anh về quê hương
Name: topic, Length: 325, dtype: object

In [10]:
def clean_topic_name(x):
    # Loại số, dấu chấm, dấu phẩy và khoảng trắng ở đầu chuỗi
    x = re.sub(r'^[\s\d\.,]+', '', x)
    # Loại dấu chấm hoặc dấu phẩy ở cuối chuỗi
    x = re.sub(r'[\.,]+$', '', x)
    return x.strip()

In [11]:
pdf_vocab['topic'] = pdf_vocab['topic'].apply(clean_topic_name)

In [12]:
pdf_vocab

,topic,english,ipa,type,vietnamese
0,chủ đề gia đình,Birth,/bɜrθ/,n,"Sự sinh ra, sự chào đời"
1,chủ đề gia đình,Brother,/ˈbrʌðər/,n,Anh/em trai
2,chủ đề gia đình,Brother-in-law,,,anh/em rể
3,chủ đề gia đình,Close,/kloʊs/,adj,"Gần gũi, gắn bó"
4,chủ đề gia đình,Cousin,/ˈkʌzən/,n,Anh chị em họ
...,...,...,...,...,...
320,Từ vựng tiếng Anh về quê hương,Lake,/leɪk/,n,Hồ
321,Từ vựng tiếng Anh về quê hương,Sea,/siː/,n,Biển
322,Từ vựng tiếng Anh về quê hương,Sand,/sænd/,n,Cát
323,Từ vựng tiếng Anh về quê hương,Valley,/ˈvæli/,n,thung lũng


In [13]:
# pdf_vocab.to_csv("vocabulary.csv", index=False, encoding="utf-8-sig")

In [14]:
pdf_vocab['topic'].value_counts()

topic
Từ vựng tiếng Anh về tính cách                        12
Từ vựng tiếng Anh chuyên ngành công nghệ thông tin    12
chủ đề môi trường                                     11
chủ đề gia đình                                       11
chủ đề du lịch                                        11
Từ vựng tiếng Anh về màu sắc                          11
Từ vựng về trường học                                 11
Từ vựng tiếng Anh về cảm xúc                          11
Từ vựng tiếng Anh chuyên ngành kế toán                11
Từ vựng tiếng Anh về nhà bếp                          11
Từ vựng về giáng sinh                                 11
Từ vựng tiếng Anh miêu tả người                       11
Từ vựng về thời trang                                 11
Từ vựng chuyên ngành marketing                        11
Từ vựng về quần áo                                    10
chủ đề nghề nghiệp                                    10
Từ vựng tiếng Anh về côn trùng                        10
Từ vựng về giao thông    

In [ ]:
# hãy thêm cột synced và cho giá trị là true 
pdf_vocab['synced'] = True

In [16]:
pdf_vocab['synced'] 

0      True
1      True
2      True
3      True
4      True
       ... 
320    True
321    True
322    True
323    True
324    True
Name: synced, Length: 325, dtype: bool

In [18]:
pdf_vocab.to_csv("vocabulary.csv", index=False, encoding="utf-8-sig")